## Taller 05: Herramientas para Indexación
##### Fecha de entrega: Lunes 13 de enero de 2025.
### Integrantes: Dilan Andrade, Hernán Sánchez y Galo Tarapués

## **Parte 1: Construcción Manual de un Índice Invertido**

1. Cargar los datos en Python.
El código utiliza la biblioteca 'pandas' para cargar el archivo CSV en un DataFrame mediante 'pd.read_csv', lo que permite manejar los datos. La función 'dataset.head()' confirma que los datos se cargaron correctamente.


In [1]:
import pandas as pd
import re
from collections import defaultdict

file_path = r'.\wiki_movie_plots_deduped.csv'
dataset = pd.read_csv(file_path)

dataset.head()


,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...


2. Construir un índice invertido manualmente:
    - Procesa las tramas de las películas para crear un índice invertido.
    - Normaliza el texto de las tramas.



El código selecciona las columnas 'Title' y 'Plot' del dataset y verifica si hay valores nulos en ellas. Luego, elimina las filas que contienen valores nulos para asegurar que los datos sean completos. Después, normaliza el texto de la columna Plot convirtiéndolo a minúsculas y eliminando la puntuación, creando una nueva columna Normalized Plot con el texto limpio.

In [3]:
data = dataset[['Title', 'Plot']]

print(f"Valores nulos antes de limpiar:\n{data.isnull().sum()}")

data = data.dropna(subset=['Title', 'Plot'])

def normalize_text(text):
    text = text.lower()  # Convertir a minúsculas
    text = re.sub(r'[^\w\s]', '', text)  # Eliminar puntuación
    return text

data['Normalized Plot'] = data['Plot'].apply(normalize_text)
data[['Title', 'Normalized Plot']].head()


Valores nulos antes de limpiar:
Title    0
Plot     0
dtype: int64


,Title,Normalized Plot
0,Kansas Saloon Smashers,a bartender is working at a saloon serving dri...
1,Love by the Light of the Moon,the moon painted with a smiling face hangs ove...
2,The Martyred Presidents,the film just over a minute long is composed o...
3,"Terrible Teddy, the Grizzly King",lasting just 61 seconds and consisting of two ...
4,Jack and the Beanstalk,the earliest known adaptation of the classic f...


Se construye un índice invertido al iterar sobre cada fila del DataFrame. Para cada película, se extrae el título y la trama normalizada, luego se tokeniza en palabras. Cada palabra se agrega al índice invertido. Luego, se eliminan los duplicados de las listas de títulos para optimizar el índice. Finalmente, se muestra una parte del índice invertido.

In [4]:
index = defaultdict(list)

for idx, row in data.iterrows():
    title = row['Title']
    plot = row['Normalized Plot']
    tokens = plot.split()  # Tokenizar el texto en palabras

    for token in tokens:
        index[token].append(title)  # Agregar el título al índice

for key in index:
    index[key] = list(set(index[key]))

list(index.items())[:10]


[('a',
  ['Alex & Emma',
   'Strip Tease Murder',
   'Carry on Jatta',
   'Shrieker',
   'Everything Is Illuminated',
   'The Attorney',
   'Sthalathe Pradhana Payyans',
   'A Cold Wind in August',
   'Margot at the Wedding',
   'En Sakhiye',
   'Ghosts',
   'East of Piccadilly',
   'Fool Coverage',
   'The Inner Circle',
   'The Pompatus of Love',
   'Dhaai Akshar Prem Ke',
   "A Dog's Purpose",
   'Flaming Brothers',
   'Damadamm!',
   "Just Another Pandora's Box",
   'Dog Pound',
   'Kanni Thaai',
   'On the Town',
   'The Legend of Qin',
   'Star Trek VI: The Undiscovered Country',
   'Ami Aaj Nasto Hoye Jai',
   'Bhagyadevatha (ഭാഗ്യദേവത)',
   'Hell in the Pacific',
   'Virginia',
   'Christmas with the Dead',
   'Nuvvostanante Nenoddantana',
   'Center Stage',
   "There's One Born Every Minute",
   'Badsha The Don',
   'Ali',
   'I Love You, Man',
   'Star!',
   'Goal!',
   'Cry for Happy',
   'Ebbtide',
   'A Daughter of the Gods',
   'Texas Across the River',
   'The Butcher, t

3. Realizar consultas en el índice invertido:
    - Implementa funciones que permitan buscar palabras clave y devolver los títulos de las películas que contienen dichas palabras.

Se implementa la función search_index, que recibe una consulta y busca cada palabra en el índice invertido. Devuelve un conjunto de títulos de películas que contienen las palabras de la consulta.

In [5]:
def search_index(query, index):
    query_tokens = normalize_text(query).split()
    results = set()

    for token in query_tokens:
        if token in index:
            results.update(index[token])  # Agregar títulos que contienen el token

    return results

# query = "saloon"
query = input("Introduce la palabra clave para la búsqueda: ")
results = search_index(query, index)
print(f"Películas encontradas para la consulta '{query}':\n{results}")


Películas encontradas para la consulta 'moon':
{'Kaalam Maari Pochu', 'Lords of Salem, TheThe Lords of Salem', 'Die Another Day', 'Isle of Forgotten Sins', 'Way...Way Out', 'Tabloid Truth', 'The Adventures of Baron Munchausen', 'The Genius Club', 'Rent', 'Wallace & Gromit: The Curse of the Were-Rabbit', 'Troublesome Night 9', 'Space Truckers', 'Project Moonbase', 'Madonna', 'The Swan Princess', 'Man in the Moon', 'Star Trek VI: The Undiscovered Country', 'The Mountain Men', 'You Ruined My Life', 'Dating the Enemy', 'Masters of the Universe', 'Kamen Rider Decade The Movie: All Riders vs. Great Shocker', ' Race to Witch Mountain', "The Mummy's Ghost", 'The Swan Princess II: Escape from Castle Mountain', 'The Adventures of Pluto Nash', 'Brick Bradford', 'Scooby-Doo and the Reluctant Werewolf', "It's In the Water", "Legend of the Guardians: The Owls of Ga'Hoole", "Jules Verne's Rocket to the Moon", 'The Ninth Configuration', 'Hugo', 'White Comanche', 'An American Werewolf in London', 'Vina

### **Parte 2: Usar Whoosh para Indexación y Recuperación**






In [6]:
!pip install whoosh


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from whoosh.index import create_in
from whoosh.fields import Schema, TEXT
from whoosh.qparser import QueryParser
import os


* create_in: Crea un índice en un directorio especificado.
* Schema y TEXT: Definen la estructura del índice, donde TEXT es el tipo de dato para almacenar texto.
* QueryParser: Convierte las consultas de búsqueda en un formato que Whoosh pueda procesar.
* os: Permite manejar operaciones del sistema de archivos, como crear directorios.

1. Configurar un índice con Whoosh:
    * Define un esquema con los campos Title y Plot.
    * Crea un índice y agrega las películas seleccionadas.

Se define un esquema con los campos title y plot, ambos como texto almacenado. Luego, se verifica si el directorio para almacenar el índice (indexdir) existe, y si no, se crea. Finalmente, se crea el índice en el directorio especificado utilizando el esquema previamente definido.

In [8]:
schema = Schema(title=TEXT(stored=True), plot=TEXT(stored=True))

indexdir = "indexdir"
if not os.path.exists(indexdir):
    os.mkdir(indexdir)

from whoosh.index import create_in
index = create_in(indexdir, schema)

print(f"Índice creado en el directorio '{indexdir}'.")


Índice creado en el directorio 'indexdir'.


2. Agregar documentos al índice:
    * Usa las funciones de escritura de Whoosh para indexar los títulos y tramas.


Se utiliza AsyncWriter de Whoosh para agregar documentos al índice. A través de un bucle, se recorren las filas de los datos, se agregan los títulos y tramas al índice mediante la función add_document. Finalmente, se confirma la adición de los documentos al índice con el método commit, y se imprime el número total de documentos agregados.

In [9]:
from whoosh.writing import AsyncWriter

writer = AsyncWriter(index)

for idx, row in data.iterrows():
    writer.add_document(title=row['Title'], plot=row['Plot'])

writer.commit()

print(f"Se han agregado {data.shape[0]} documentos al índice.")


Se han agregado 34886 documentos al índice.


3. Realizar consultas:
    * Busca películas relevantes usando palabras clave relacionadas con las tramas (e.g., "dinosaurs", "cyborg").


Se define una función 'search_whoosh' para realizar consultas en el índice. Esta función abre el índice para lectura, analiza la consulta usando el QueryParser con el campo plot, y luego busca los documentos que coincidan con la consulta. Los resultados de la búsqueda se almacenan y se extraen los títulos de las películas correspondientes. Finalmente, se realiza una consulta y se muestran las películas encontradas.

In [10]:
def search_whoosh(query, index):
    with index.searcher() as searcher:
        query_parser = QueryParser("plot", index.schema)
        parsed_query = query_parser.parse(query)
        results = searcher.search(parsed_query)
        titles = [result['title'] for result in results]
        return titles

# query = "dinosaurs park"
query = input("Introduce la palabra clave para la búsqueda: ")
results = search_whoosh(query, index)
print(f"Películas encontradas para la consulta '{query}':\n{results}")


Películas encontradas para la consulta 'moon':
["Goin' South", 'Love by the Light of the Moon', 'Andrew Lau, Alan Mak', "This Girl's Life", 'Set Up', 'At Play in the Fields of the Lord', 'Missile to the Moon', 'Kamen Rider World', 'Pontiac Moon', 'Nude on the Moon']


Para hacer mas pruebas, se define una lista de consultas con diferentes palabras clave relacionadas con las tramas de las películas. Luego, se recorre esta lista y se realiza una búsqueda para cada palabra clave utilizando la función search_whoosh. Los resultados de cada consulta se imprimen, mostrando las películas encontradas para cada término de búsqueda.

In [11]:
queries = ["cyborg", "time travel", "space adventure", "robot"]
for query in queries:
    results = search_whoosh(query, index)
    print(f"Películas encontradas para '{query}': {results}\n")


Películas encontradas para 'cyborg': ['Future War', 'Space Truckers', 'JAKQ Dengeki Tai', 'JAKQ Dengeki Tai vs. Goranger', 'Kung Fu Cyborg', 'Future X-Cops', 'Kamen Rider V3', 'Kamen Rider V3 vs. the Destron Monsters', 'Cyborg She', 'Cyborg She']

Películas encontradas para 'time travel': ['Time Chasers', 'Love Story 2050', 'Timeranger vs. GoGoV', 'Abby Sen', 'Dimensions', 'Millennium', 'Primer', 'The Man with Rain in His Shoes', 'The Road', 'About Time']

Películas encontradas para 'space adventure': ['Space Ship Sappy', 'Moonshine', 'Galaxy Quest', 'Time Bandits', 'Looney Tunes: Back in Action', 'Gokaiger Goseiger Super Sentai 199 Hero Great Battle', 'Gold Diggers: The Secret of Bear Mountain', 'Aliens in the Attic', 'Until the End of the World (a.k.a. Bis ans Ende der Welt)', 'African Cats']

Películas encontradas para 'robot': ["Daft Punk's Electroma", 'Robosapien: Rebooted', 'Tobor the Great', 'Hardware', 'Chopping Mall', 'Doraemon: Nobita and the New Steel Troops: ~Angel Wings~',

Finalemente, para hacer comprobaciones adicionales, se utiliza un buscador del índice para obtener la cantidad total de documentos presentes en el índice. Con el método doc_count(), se imprime el número de documentos almacenados en el índice.

In [12]:
# Validar cuántos documentos hay en el índice
with index.searcher() as searcher:
    print(f"El índice contiene {searcher.doc_count()} documentos.")


El índice contiene 34886 documentos.


## **Parte 3: Usar Elasticsearch para Indexación y Recuperación**
1. Iniciar Elasticsearch con Docker.


- Descarga Docker Desktop desde docker.com.
- Descargar e iniciar Elasticsearch|



    docker run -d --name elasticsearch -p 9200:9200 -e "discovery.type=single-node" docker.elastic.co/elasticsearch/elasticsearch:8.10.2


*-d: Ejecuta el contenedor en segundo plano.*

*--name ela|sticsearch: Nombra al contenedor como elasticsearch.*

*-p 9200:9200: Expone el puerto 9200 del contenedor a tu máquina local.*

*-e "discovery.type=single-node": Configura Elasticsearch en modo de nodo único (ideal para pruebas locales).*

Verifica que Elasticsearch esté corriendo visitando:

    http://localhost:9200

2. Configurar el cliente Elasticsearch en Python:
    * Conecta Elasticsearch y crea un índice con los campos Title y Plot.

Instala la librería de Python para conectarte a Elasticsearch:

    pip install elasticsearch


Luego, configura el cliente en tu código:

In [ ]:
from elasticsearch import Elasticsearch

# Conexión a Elasticsearch
es = Elasticsearch("http://localhost:9200")

# Verificar la conexión
if es.ping():
    print("Conectado a Elasticsearch")
else:
    print("No se pudo conectar a Elasticsearch")

3. Indexar documentos:
    * Inserta cada película en el índice de Elasticsearch, usando el título como identificador.


Define el esquema para el índice y créalo. Aquí definimos los campos Title y Plot:

In [ ]:
# Crear un índice con el esquema especificado
index_name = "movies"
if not es.indices.exists(index=index_name):
    es.indices.create(
        index=index_name,
        body={
            "mappings": {
                "properties": {
                    "Title": {"type": "text"},
                    "Plot": {"type": "text"},
                }
            }
        },
    )
    print(f"Índice '{index_name}' creado.")
else:
    print(f"Índice '{index_name}' ya existe.")


Inserta cada película en el índice de Elasticsearch, utilizando el título como identificador único:

In [ ]:
# Indexar las películas en Elasticsearch
for idx, row in data.iterrows():
    doc = {"Title": row["Title"], "Plot": row["Plot"]}
    es.index(index=index_name, id=idx, body=doc)

print(f"Se han indexado {len(data)} documentos en el índice '{index_name}'.")


4. Realizar consultas:
    * Realiza búsquedas utilizando tramas relacionadas con palabras clave como "time travel" o "genetic engineering".

# Función para buscar en Elasticsearch
def search_elasticsearch(query, index):
    response = es.search(
        index=index,
        body={
            "query": {
                "match": {
                    "Plot": query  # Busca en el campo 'Plot'
                }
            }
        },
    )
    # Extraer títulos de los resultados
    titles = [hit["_source"]["Title"] for hit in response["hits"]["hits"]]
    return titles

# Ejemplo de consulta
query = input("Introduce la palabra clave para la búsqueda: ")
results = search_elasticsearch(query, index_name)
print(f"Películas encontradas para '{query}': {results}")


Codigo completo del archivo .py:

In [ ]:
from elasticsearch import Elasticsearch
import pandas as pd
import re

# Conexión a Elasticsearch
def connect_elasticsearch():
    es = Elasticsearch("http://localhost:9200")
    if es.ping():
        print("Conectado a Elasticsearch")
        return es
    else:
        print("No se pudo conectar a Elasticsearch")
        exit()

# Crear un índice con esquema
def create_index(es, index_name):
    if not es.indices.exists(index=index_name):
        es.indices.create(
            index=index_name,
            body={
                "mappings": {
                    "properties": {
                        "Title": {"type": "text"},
                        "Plot": {"type": "text"},
                    }
                }
            },
        )
        print(f"Índice '{index_name}' creado.")
    else:
        print(f"Índice '{index_name}' ya existe.")

# Normalizar texto
def normalize_text(text):
    text = text.lower()
    text = re.sub(r"[^\w\s]", "", text)
    return text

# Indexar documentos
def index_documents(es, index_name, data):
    for idx, row in data.iterrows():
        doc = {"Title": row["Title"], "Plot": row["Plot"]}
        es.index(index=index_name, id=idx, body=doc)
    print(f"Se han indexado {len(data)} documentos en el índice '{index_name}'.")

# Realizar búsqueda
def search_elasticsearch(es, query, index_name):
    response = es.search(
        index=index_name,
        body={
            "query": {
                "match": {
                    "Plot": query
                }
            }
        },
    )
    titles = [hit["_source"]["Title"] for hit in response["hits"]["hits"]]
    return titles

# Cargar datos del CSV
def load_data(file_path):
    dataset = pd.read_csv(file_path)
    data = dataset[["Title", "Plot"]].dropna()
    data["Plot"] = data["Plot"].apply(normalize_text)
    return data

# Validar documentos en el índice
def validate_documents(es, index_name):
    doc_count = es.count(index=index_name)["count"]
    print(f"El índice '{index_name}' contiene {doc_count} documentos.")

if __name__ == "__main__":
    # Configuración
    INDEX_NAME = "movies"
    FILE_PATH = "./wiki_movie_plots_deduped.csv"

    # 1. Conectar a Elasticsearch
    es = connect_elasticsearch()

    # 2. Crear índice
    create_index(es, INDEX_NAME)

    # 3. Cargar datos
    data = load_data(FILE_PATH)

    # 4. Indexar documentos
    index_documents(es, INDEX_NAME, data)

    # 5. Validar cantidad de documentos
    validate_documents(es, INDEX_NAME)

    # 6. Realizar consultas
    queries = ["time travel", "genetic engineering", "space adventure", "robot"]
    for query in queries:
        results = search_elasticsearch(es, query, INDEX_NAME)
        print(f"Resultados para '{query}': {results}")

### Reflexiones:


##### 1. Implementación de un índice invertido manual

Al trabajar con un índice invertido manual, enfrentamos desafíos relacionados con la normalización del texto. A pesar de usar expresiones regulares para eliminar puntuación y convertir texto a minúsculas, notamos que algunas palabras compuestas o contracciones no se manejaban correctamente. Aprendimos que, al procesar datos a gran escala, es crucial optimizar la normalización y tener en cuenta las posibles variaciones en el texto para asegurar la precisión del índice. 

##### 2. Uso de Whoosh para indexación y recuperación

El uso de Whoosh mejoró significativamente la indexación. Sin embargo, al integrar Whoosh, tuvimos que prestar especial atención al esquema de indexación y al proceso de búsqueda, ya que las consultas no siempre eran precisas al principio. Aprendimos la importancia de ajustar bien los parámetros de tokenización y análisis para obtener resultados exactos. La experiencia nos enseñó cómo trabajar con índices optimizados en entornos de búsqueda de texto completo.

##### 3. Conexión y uso de Elasticsearch

La configuración de Elasticsearch fue más compleja de lo esperado, especialmente al trabajar con Docker y configurar la conexión desde Python. A pesar de los desafíos iniciales, una vez conectados, la creación de índices y la indexación de documentos fueron rápidas. Aprendimos que la clave para un rendimiento óptimo en Elasticsearch es definir correctamente los mapeos del índice. La escalabilidad y eficiencia de Elasticsearch fueron notables, superando las soluciones anteriores, y al final, conseguimos una solución robusta y eficiente para manejar grandes volúmenes de datos.

En general, la integración de Elasticsearch fue un paso decisivo que mejoró tanto la velocidad como la precisión de las búsquedas, consolidando nuestra solución de indexación.